In [ ]:
#Stage 1 (STT)
# Import necessary libraries for Speech to Text
import neuralspace as ns
FileName='/content/audio.ogg'
# Configure the API settings and file details
vai = ns.VoiceAI(api_key='your_api_key')
config = {
    'file_transcription': {
        'language_id': 'ar-ms',
        'mode': 'advanced'
    }
}

# Create a new file transcription job
job_id = vai.transcribe(file=FileName, config=config)
print(f'Created job: {job_id}')

# Check the job's status and wait for completion
result = vai.get_job_status(job_id)
print(f'Current status: {result}')
print('Waiting for completion...')
result = vai.poll_until_complete(job_id)
print(result)
text_result = result['data']['result']['transcription']['transcript']


In [ ]:
#Stage 2 (TTT)
# Import necessary libraries for Text to Text Processing
import textwrap
from IPython.display import display, Markdown

# Define helper function for formatting text to Markdown
def to_markdown(text):
    text = text.replace('.', ' *')
    return Markdown(textwrap.indent(text, ' ', predicate=lambda _: True))

# Sample text transformation (update as per the requirements)
input_text = text_result
prompt_text = f"You will be provided with text. Remove any repeated words or phrases that result from the stutter. Adjust the output: {input_text}"
model_id = 'gemini-pro'  # specify the model id if needed

# Assuming a function to call a generative model is defined (e.g., generative_model.generate)
response = generative_model.generate(prompt_text)
fixed_text = response.text  # Extracting processed text from the model's response


In [ ]:
#Stage 3 (TTS)
# Import necessary libraries for Text to Speech
import requests

# Configure the ElevenLabs API for text to speech
elevenlabs_api_key = 'your_elevenlabs_api_key'
client = ElevenLabs(api_key=elevenlabs_api_key)
voice = client.clone(
    name="Sara",
    description="A 20-year-old woman discusses her daily routine.",
    files=[FileName]
)

# Generate speech from text
voice_id = voice.voice_id
url = f"https://api.elevenlabs.io/v1/text-to-speech/{voice_id}"
headers = {
    "Accept": "audio/mpeg",
    "Content-Type": "application/json",
    "xi-api-key": elevenlabs_api_key
}
data = {
    "text": fixed_text,
    "model_id": "eleven_multilingual_v2",
    "voice_settings": {
        "stability": 0.5,
        "similarity_boost": 0.75
    }
}
response = requests.post(url, json=data, headers=headers)

# Save the audio output
output_file = 'output.mp3'
with open(output_file, 'wb') as f:
    for chunk in response.iter_content(chunk_size=1024):
        if chunk:
            f.write(chunk)

print(f"Audio output saved as {output_file}")
